In [1]:
import findspark
findspark.init('/home/shashank/spark-2.3.2-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_codealong').getOrCreate()
from pyspark.ml.regression import LinearRegression

In [2]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)

In [3]:
data.createOrReplaceTempView('lr_sql')

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [11]:
for i in data.head():
    print(i)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [13]:
#Import Vector Assembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [14]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [16]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'], \
                            outputCol= 'features')

In [17]:
output = assembler.transform(data)

In [18]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
output = output.withColumnRenamed('Yearly Amount Spent', 'Yearly_Amount_Spent')
output.head() #notice the feature option

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly_Amount_Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

# Select final data

In [25]:
output.createOrReplaceTempView('lr_sql')

In [26]:
final_data = spark.sql("FROM lr_sql SELECT features, Yearly_Amount_Spent")

In [27]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly_Amount_Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [43]:
train,test = final_data.randomSplit([0.7,0.3])

In [44]:
train.count()

341

In [45]:
test.count()

159

In [46]:
print(train.describe().show())
print(test.describe().show())

+-------+-------------------+
|summary|Yearly_Amount_Spent|
+-------+-------------------+
|  count|                341|
|   mean| 500.84933310830627|
| stddev|  83.09629814253438|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+

None
+-------+-------------------+
|summary|Yearly_Amount_Spent|
+-------+-------------------+
|  count|                159|
|   mean| 496.02136188278575|
| stddev|  70.66145964592243|
|    min|  319.9288698031936|
|    max|  700.9170916173961|
+-------+-------------------+

None


In [47]:
lr = LinearRegression(featuresCol='features', labelCol='Yearly_Amount_Spent', predictionCol='prediction') 
#If the column lables are are the same as default, you can also not specify them

In [48]:
lr_model = lr.fit(train)

In [49]:
test_results = lr_model.evaluate(test)

In [52]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-12.318021048419155|
|-6.2984516150245895|
|-21.528305756642226|
| -7.915118954902027|
|-4.1464128889110725|
| 22.155814575927707|
|  18.38825019157781|
| -4.641247617189492|
|  -9.80571011419255|
|-18.088495514054102|
|-14.610401076798098|
|  17.69784907242888|
| -7.056763730668422|
|  3.132330791290144|
|-10.791829878162616|
|-18.954693362755336|
|-1.9644175310699552|
| -2.867817343084198|
| -5.490173316950802|
| 12.131880263628886|
+-------------------+
only showing top 20 rows



In [57]:
print("R-square: {}".format(test_results.r2))
print("RMSE: {}".format(test_results.rootMeanSquaredError))

R-square: 0.977703971177703
RMSE: 10.517834776387582


In [58]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly_Amount_Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



Mean = 499.314, RMSE = 10.518

In [59]:
#Deploy model
test.createOrReplaceTempView('test_sql')
unlab = spark.sql("FROM test_sql SELECT features")

In [61]:
unlab.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[31.0613251567161...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.2681042107507...|
|[31.2834474760581...|
|[31.3123495994443...|
|[31.4252268808548...|
|[31.5261978982398...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.6005122003032...|
|[31.7207699002873...|
|[31.7366356860502...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8186165667690...|
|[31.8530748017465...|
|[31.8745516945853...|
|[31.9096268275227...|
+--------------------+
only showing top 20 rows



In [62]:
predictions = lr_model.transform(unlab)

In [63]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|332.24689085161276|
|[31.0613251567161...| 493.8539096729262|
|[31.1239743499119...|  508.475359596408|
|[31.1280900496166...| 565.1678057019567|
|[31.2681042107507...|  427.616946062735|
|[31.2834474760581...| 569.6252748497398|
|[31.3123495994443...| 445.2031678363628|
|[31.4252268808548...| 535.4079662719514|
|[31.5261978982398...|418.90023630653036|
|[31.5702008293202...|  564.033987655459|
|[31.5741380228732...|  559.019673237385|
|[31.6005122003032...|461.47500241866805|
|[31.7207699002873...| 545.8316972086914|
|[31.7366356860502...| 493.8011154642418|
|[31.8093003166791...| 547.5637292410038|
|[31.8164283341993...| 520.0771848664117|
|[31.8186165667690...| 448.3830909012056|
|[31.8530748017465...| 462.1529408054362|
|[31.8745516945853...| 397.7754175632183|
|[31.9096268275227...| 551.3141554096103|
+--------------------+------------